# import

In [65]:
import pandas as pd 
import numpy as np

# file save
import json 

# regular expression
import re 

# selenium

In [66]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException

In [67]:
scrape_list = []

In [68]:
for series, urls in links.items():
    print(f"Series: {series}")
    for url in urls:
        print(f"URL: {url}")
        scrape_list.append(url)

Series: Original_Series
URL: https://bulbapedia.bulbagarden.net/wiki/Base_Set_(TCG)#Set_lists
URL: https://bulbapedia.bulbagarden.net/wiki/Jungle_(TCG)#Set_lists
URL: https://bulbapedia.bulbagarden.net/wiki/Fossil_(TCG)#Set_lists
URL: https://bulbapedia.bulbagarden.net/wiki/Base_Set_2_(TCG)#Set_lists
URL: https://bulbapedia.bulbagarden.net/wiki/Team_Rocket_(TCG)#Set_lists
URL: https://bulbapedia.bulbagarden.net/wiki/Gym_Heroes_(TCG)#Set_lists
URL: https://bulbapedia.bulbagarden.net/wiki/Gym_Challenge_(TCG)#Set_lists
Series: Neo_Series
URL: https://bulbapedia.bulbagarden.net/wiki/Neo_Genesis_(TCG)#Set_lists
URL: https://bulbapedia.bulbagarden.net/wiki/Neo_Discovery_(TCG)#Set_lists
URL: https://bulbapedia.bulbagarden.net/wiki/Neo_Revelation_(TCG)#Set_lists
URL: https://bulbapedia.bulbagarden.net/wiki/Neo_Destiny_(TCG)#Set_lists
Series: Legendary_Collection_Series
URL: https://bulbapedia.bulbagarden.net/wiki/Legendary_Collection_(TCG)#Set_lists
Series: e-Card_Series
URL: https://bulbapedi

In [70]:
len(scrape_list)

110

In [71]:
# File path to the JSON file
file_path = "pokemonTCG_links.json"

# Open the JSON file and load its contents into a variable
with open(file_path, 'r') as file:
    data = json.load(file)

In [72]:
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode

# Set up Chrome driver service
service = Service('/path/to/chromedriver')  # Replace with the actual path to chromedriver


# Create an empty list to store the data
all_data = []

# Iterate over each series in the data
for series, urls in data.items():
    print(f"Series: {series}")
    for url in urls:
        print(f"URL: {url}")

        # Start the Chrome webdriver
        driver = webdriver.Chrome(service=service, options=chrome_options)

        # Load the webpage
        driver.get(url)

        # Wait for the page to load (you may need to adjust the waiting time depending on the page)
        driver.implicitly_wait(10)  # Wait for 10 seconds

        xPath = '//*[@id="mw-content-text"]/div/table[4]/tbody/tr/td[1]/table'
        # Find the table element using the XPath
        table_element = driver.find_element(By.XPATH, xPath)

        # Get the HTML source of the table element
        table_html = table_element.get_attribute('innerHTML')

        # Create a BeautifulSoup object from the HTML source
        soup = BeautifulSoup(table_html, 'html.parser')

        # Find the serial numbers from a table/html
        numbers = soup.find_all('td', attrs={'align': 'center', 'style': 'background:#FFFFFF'})
        numbers_filtered = [num.text.strip() for num in numbers if re.match(r".*\d+/\d+$", num.text.strip())]

        if numbers_filtered == []:
            print("empty list")
            xPath = '//*[@id="mw-content-text"]/div/table[5]/tbody/tr/td[1]/table'

            # Find the table element using the XPath
            table_element = driver.find_element(By.XPATH, xPath)

            # Get the HTML source of the table element
            table_html = table_element.get_attribute('innerHTML')

            # Create a BeautifulSoup object from the HTML source
            soup = BeautifulSoup(table_html, 'html.parser')

            # Find the serial numbers from a table/html
            numbers = soup.find_all('td', attrs={'align': 'center', 'style': 'background:#FFFFFF'})
            numbers_filtered = [num.text.strip() for num in numbers if re.match(r".*\d+/\d+$", num.text.strip())]

        # Find card names through HTML
        items = soup.find_all('td', attrs={'style': 'background:#FFFFFF'})
        cardNames = []
        for item in items:
            if item.find('a') and '/100' not in item.text:
                card_name = item.find('a').text.strip()
                cardNames.append(card_name)
        cardNames = list(filter(lambda x: x != '', cardNames))

        # Find types
        types = soup.find_all('th', attrs={'align': 'center'})

        pTypes = []
        pattern = r'^[A-Z]'

        for i, html in enumerate(types):
            # Determine the appropriate extraction method based on a condition
            html1 = html.text

            if re.search(pattern, str(html1)):
                # Extract method 1
                title = str(html1).strip()
                pTypes.append(title)
            else:
                # Ensure there are at least two 'a' tags for energy titles
                a_tags = html.find_all('a')
                num_a_tags = len(a_tags)

                if num_a_tags >= 2:
                    type1 = a_tags[0]
                    title1 = type1['title']

                    type2 = a_tags[1]
                    title2 = type2['title']

                    pTypes.append([title1, title2])  # Append a list of two types to the main list
                elif num_a_tags == 1:
                    a_tag = a_tags[0]
                    title = a_tag['title']

                    pTypes.append(title)  # Append single type to the main list
                else:
                    print(f"Error: Insufficient 'a' tags for element {i}")

        image_anchors = soup.find_all('a', class_='image')
        possible_titles = ['Rare Holo', 'Rare', 'Uncommon', 'Common', 'Rare Holo ex', 'Shiny Rare', 'ShinyRare Holo']
        Rarity = []
        for anchor in image_anchors:
            title = anchor.get('title')
            if title in possible_titles:
                Rarity.append(title)

        # Fix data length
        if len(Rarity) < len(numbers_filtered):
            diff = len(numbers_filtered) - len(Rarity)
            Rarity.extend(['NULL'] * diff)

        extracted_string = url.split('/')[-2].replace('_', ' ')
        extracted_string = re.sub(r'\([^)]*\)', '', extracted_string).strip()  # Remove parentheses and contents

        # Create a dictionary for the extracted data
        data = []
        for i in range(0, len(numbers_filtered)):
            dNum = numbers_filtered[i]
            dName = cardNames[i]
            dType = pTypes[i]
            dRarity = Rarity[i]
            
            ######
            string = url

            match = re.search(r'\/([^\/]+)\(TCG\)', string)
            if match:
                extracted_string = match.group(1).replace('_', ' ')
    
            
            ######
            data.append({
                'serial_number': dNum,
                'name': dName,
                'type': dType,
                'rarity': dRarity,
                'set': extracted_string
            })

        # Append the data to the main list
        all_data.extend(data)
        
        # Close the Chrome webdriver
        driver.quit()

# Create a dataframe from the collected data
data_df = pd.DataFrame(all_data, columns=['serial_number', 'name', 'type', 'rarity', 'set'])

# Print the resulting dataframe
print(data_df)


Series: Original_Series
URL: https://bulbapedia.bulbagarden.net/wiki/Base_Set_(TCG)#Set_lists
empty list
URL: https://bulbapedia.bulbagarden.net/wiki/Jungle_(TCG)#Set_lists
empty list
URL: https://bulbapedia.bulbagarden.net/wiki/Fossil_(TCG)#Set_lists
empty list
URL: https://bulbapedia.bulbagarden.net/wiki/Base_Set_2_(TCG)#Set_lists
empty list
URL: https://bulbapedia.bulbagarden.net/wiki/Team_Rocket_(TCG)#Set_lists
empty list
URL: https://bulbapedia.bulbagarden.net/wiki/Gym_Heroes_(TCG)#Set_lists
URL: https://bulbapedia.bulbagarden.net/wiki/Gym_Challenge_(TCG)#Set_lists
empty list
Series: Neo_Series
URL: https://bulbapedia.bulbagarden.net/wiki/Neo_Genesis_(TCG)#Set_lists
URL: https://bulbapedia.bulbagarden.net/wiki/Neo_Discovery_(TCG)#Set_lists
empty list
URL: https://bulbapedia.bulbagarden.net/wiki/Neo_Revelation_(TCG)#Set_lists
empty list
URL: https://bulbapedia.bulbagarden.net/wiki/Neo_Destiny_(TCG)#Set_lists
empty list
Series: Legendary_Collection_Series
URL: https://bulbapedia.bul

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="mw-content-text"]/div/table[4]/tbody/tr/td[1]/table"}
  (Session info: headless chrome=114.0.5735.106)
Stacktrace:
0   chromedriver                        0x00000001099046b8 chromedriver + 4937400
1   chromedriver                        0x00000001098fbb73 chromedriver + 4901747
2   chromedriver                        0x00000001094b9616 chromedriver + 435734
3   chromedriver                        0x00000001094fce0f chromedriver + 712207
4   chromedriver                        0x00000001094fd0a1 chromedriver + 712865
5   chromedriver                        0x000000010953e9a4 chromedriver + 981412
6   chromedriver                        0x000000010952103d chromedriver + 860221
7   chromedriver                        0x000000010953be76 chromedriver + 970358
8   chromedriver                        0x0000000109520de3 chromedriver + 859619
9   chromedriver                       

In [74]:
looksie = pd.DataFrame(all_data)

In [82]:
import sketch

In [83]:
looksie.sketch.ask('how to print a list of values in the column set')

In [87]:
sets = looksie['set'].unique().tolist()

In [107]:
len(sets)

31

In [104]:
for i in sets:
    
    print(i)
    print(looksie[looksie['set'] == i])
    print('\n')
    print("************")
    
    

Base Set 
    serial_number              name       type     rarity        set
0           1/102          Alakazam    Psychic  Rare Holo  Base Set 
1           2/102         Blastoise      Water  Rare Holo  Base Set 
2           3/102           Chansey  Colorless  Rare Holo  Base Set 
3           4/102         Charizard       Fire  Rare Holo  Base Set 
4           5/102          Clefairy  Colorless  Rare Holo  Base Set 
..            ...               ...        ...        ...        ...
97         98/102       Fire Energy       Fire       NULL  Base Set 
98         99/102      Grass Energy      Grass       NULL  Base Set 
99        100/102  Lightning Energy  Lightning       NULL  Base Set 
100       101/102    Psychic Energy    Psychic       NULL  Base Set 
101       102/102      Water Energy      Water       NULL  Base Set 

[102 rows x 5 columns]


************
Jungle 
    serial_number        name       type     rarity      set
102          1/64    Clefable  Colorless  Rare Holo  J

In [110]:
for i in sets:
    
    print(i)
    print(len(looksie[looksie['set'] == i]))
    print('\n')
    print("************")
    
    

Base Set 
102


************
Jungle 
64


************
Fossil 
62


************
Team Rocket 
83


************
Gym Heroes 
132


************
Gym Challenge 
132


************
Neo Genesis 
111


************
Expedition Base Set 
165


************
EX Ruby & Sapphire 
109


************
EX Sandstorm 
100


************
EX Dragon 
100


************
EX Team Magma vs Team Aqua 
97


************
EX Hidden Legends 
102


************
EX FireRed & LeafGreen 
116


************
EX Team Rocket Returns 
111


************
EX Deoxys 
108


************
EX Unseen Forces 
117


************
EX Delta Species 
114


************
EX Legend Maker 
93


************
EX Holon Phantoms 
111


************
EX Crystal Guardians 
100


************
EX Dragon Frontiers 
101


************
EX Power Keepers 
108


************
Diamond & Pearl 
130


************
Mysterious Treasures 
124


************
Secret Wonders 
132


************
Legends Awakened 
146


************
Stormfront 
103


************
Plat

In [ ]:
# sets list only holds 1/4 of the actual sets. the scrape method seemed to skip over a few. maybe we can do process of elemination by deleting the links that were successful and 